# Projeto AceleraDev Data Science (Junho de 2020)
<sub>
    Feito por Vini Antunes
</sub>
<br>
<ul>
    <li><a href="https://www.linkedin.com/in/vini-antunes/"> LinkedIn </a></li>
    <li><a href="https://github.com/ViniViniAntunes"> Github </a></li>
    <li><a href="vini.antunes2705@gmail.com"> Email </a></li>
</ul>


# Objetivo
<p>
    O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).
</p>

# Contextualização
<p>
    Algumas empresas gostariam de saber quem são as demais empresas em um determinado mercado (população) que tem maior probabilidade se tornarem seus próximos clientes. Ou seja, a sua solução deve encontrar no mercado quem são os leads mais aderentes dado as características dos clientes presentes no portfólio do usuário.
</p>

<p>
    Além disso, sua solução deve ser agnóstica ao usuário. Qualquer usuário com uma lista de clientes que queira explorar esse mercado pode extrair valor do serviço.
</p>

### __Para o desafio, deverão ser consideradas as seguintes bases:__
<ul>
    <li> Mercado: Base com informações sobre as empresas do Mercado a ser considerado. </li>
    <li> Portfolio 1: Ids dos clientes da empresa 1 </li>
    <li> Portfolio 2: Ids dos clientes da empresa 2 </li>
    <li> Portfolio 3: Ids dos clientes da empresa 3 </li>
</ul>

**Obs: todas as empresas(ids) dos portfolios estão contidos no Mercado(base de população).**

<p>
    Link para download das bases Mercado, Portfolio 1, Portfolio 2 e Portfolio 3 respectivamente:
    <ul>
        <li><a href="https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_market.csv.zip"> estaticos_market.csv.zip </a></li>
        <li><a href="https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio1.csv"> estaticos_portfolio1.csv </a></li>
        <li><a href="https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio2.csv"> estaticos_portfolio2.csv </a></li>
        <li><a href="https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_portfolio3.csv"> estaticos_portfolio3.csv </a></li>
    </ul>
</p>

<p>
    As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.
</p>

# Descrição de variáveis

**Requisitos técnicos obrigatórios**
<p>
    <ul>
        <li> Utilizar técnicas de data science e machine learning para desenvolver o projeto;</li>
        <li> Apresentar o desenvolvimento e outputs do modelo em um Jupyter Notebook ou outra tecnologia de apresentação de Output de modelos de Machine Learning; </li>
        <li> A análise deve considerar os seguintes pontos: análise exploratória dos dados, tratamento dos dados, avaliação de algoritmos, treinamento do modelo, avaliação de performance do modelo e visualização dos resultados; </li>
        <li> Para a apresentação do projeto, o tempo entre o treinamento do modelo e o output deve ser menor que 20 min. </li>
    </ul>
</p>

# Análise exploratória dos dados

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
#Baixando os arquivos csv
market = pd.read_csv('data\\estaticos_market.csv')
portifolio1 = pd.read_csv('data\\estaticos_portfolio1.csv')
portifolio2 = pd.read_csv('data\\estaticos_portfolio2.csv')
portifolio3 = pd.read_csv('data\\estaticos_portfolio3.csv')

In [3]:
portifolio1.drop('Unnamed: 0', axis=1, inplace=True)
portifolio2.drop('Unnamed: 0', axis=1, inplace=True)
portifolio3.drop('Unnamed: 0', axis=1, inplace=True)
market.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# Mostrando o tamanho do dataset market
market.shape

(462298, 181)

**Como o dataset <code>market</code> é muito grande, vamos fazer essa limpeza primeiramente no dataset <code>portifolio1</code> que contém as mesmas colunas e depois faremos dataset de interesse**

In [5]:
# Mostrando o tamanho do dataset portifolio1 antes da limpeza
portifolio1.shape

(555, 181)

# Tratamento dos dados

# Parte 1 - Retirando o excesso

**Para começar, vamos retirar as variáveis que contém valores únicos. Pois acreditamos que sem variabilidade nos dados de determinada variável, não será possível diferenciar as entradas com isso.**

## Retirando as colunas com valores únicos

In [6]:
#Definindo uma função que retira as colunas com valores únicos
def retira_colunas_de_valores_unicos(df):

    # Laço para iterar em todos as colunas
    for coluna in df.columns:

        # Selecionando as colunas de interesse
        if df[coluna].nunique() <= 1:

            # Retirando as colunas selecionadas
            df.drop(coluna, axis=1, inplace=True)
    
    # Retornando um DataFrame sem colunas com valores únicos
    return df

In [7]:
# Aplicando a função recém criada no dataset portifolio1 e 
#   armazenando o seu retorno na própria variável
portifolio1 = retira_colunas_de_valores_unicos(portifolio1)

# Mostrando o tamanho do dataset portifolio1 depois de retiras os valores únicos
portifolio1.shape

(555, 143)

**Agora, nas colunas do tipo "boolean", vamos trocar <code>False</code> e <code>True</code> por <code>0</code> e <code>1</code> respectivamente.**

## Trocando bool para 0 e 1

In [8]:
# Mostrando as colunas com valores do tipo "boolean" antes da troca
[coluna for coluna in portifolio1.dtypes[portifolio1.dtypes == 'bool'].index]

['fl_matriz',
 'fl_me',
 'fl_sa',
 'fl_mei',
 'fl_email',
 'fl_telefone',
 'fl_antt',
 'fl_veiculo',
 'fl_passivel_iss']

In [9]:
# Definindo uma função que troca "bool" para "int"
def troca_bool_para_numero(df):

    # Selecionando as colunas de interesse
    colunas_bool = [coluna for coluna in df.dtypes[df.dtypes == 'bool'].index]
    
    # Laço para iterar em todas as colunas de interesse
    for coluna in colunas_bool:

        # Realizando a troca
        df[coluna] = [1 if valor == True else 0 for valor in df[coluna]]
    
    # Retornando um DataFrame com "int" no lugar de "bool"
    return df

In [10]:
# Aplicando a função recém criada no dataset portifolio1 e 
#   armazenando o seu retorno na própria variável
portifolio1 = troca_bool_para_numero(portifolio1)

# Mostrando as colunas com valores do tipo "boolean" depois da troca
[coluna for coluna in portifolio1.dtypes[portifolio1.dtypes == 'bool'].index]

[]

**Com isso feito, vamos trocar as variáveis do tipo <code>int64</code> e <code>float64</code> para <code>int32</code> e <code>float32</code> respectivamente. Com isso vamos reduzir um pouco a memória utilizada ao rodar os nossos modelos sem perda de precisão significativa para o nosso objetivo.**

## Trocando de 64 para 32

In [11]:
# Definindo uma função para trocar para um tipo de variável que usa menos memória
def reduz_memoria(df):

    # Criando duas listas para armazenar as colunas que selecionaremos
    colunas_int = []
    colunas_float = []

    # Laço para iterar em todas as colunas com dados do tipo "int64" e "float64"
    for coluna, tipo in enumerate(df.dtypes):

        # Selecionando as colunas com dados do tipo "int64"
        if tipo == 'int64':

            # Amazenando os nomes das colunas do tipo "int64"
            colunas_int.append(df.dtypes.index[coluna])

            # Realizando a troca para "int16"
            df[colunas_int] = df[colunas_int].astype(np.int16)
        
        # Selecionando as colunas com dados do tipo "int64"
        elif tipo == 'float64':

            # Amazenando os nomes das colunas do tipo "float64"
            colunas_float.append(df.dtypes.index[coluna])

            # Realizando a troca para "float32"
            df[colunas_float] = df[colunas_float].astype('float32')
    
    # Retornando um DataFrame com "uint8" e "float32"
    return df

In [12]:
# Mostrando as colunas com valores do tipo "int64" antes da troca
portifolio1.dtypes[portifolio1.dtypes=='int64']

fl_matriz                          int64
fl_me                              int64
fl_sa                              int64
fl_mei                             int64
fl_email                           int64
fl_telefone                        int64
fl_antt                            int64
fl_veiculo                         int64
vl_total_veiculos_pesados_grupo    int64
vl_total_veiculos_leves_grupo      int64
fl_passivel_iss                    int64
qt_filiais                         int64
dtype: object

In [13]:
# Mostrando as colunas com valores do tipo "float64" antes da troca
portifolio1.dtypes[portifolio1.dtypes=='float64']

idade_empresa_anos           float64
vl_total_veiculos_leves      float64
vl_total_veiculos_pesados    float64
qt_art                       float64
vl_potenc_cons_oleo_gas      float64
                              ...   
qt_funcionarios_12meses      float64
qt_funcionarios_24meses      float64
tx_crescimento_12meses       float64
tx_crescimento_24meses       float64
tx_rotatividade              float64
Length: 110, dtype: object

In [14]:
# Aplicando a função recém criada no dataset portifolio1 e 
#   armazenando o seu retorno na própria variável
portifolio1 = reduz_memoria(portifolio1)

In [15]:
# Mostrando as colunas com valores do tipo "int64" depois da troca
portifolio1.dtypes[portifolio1.dtypes == 'int64']

Series([], dtype: object)

In [16]:
# Mostrando as colunas com valores do tipo "float64" depois da troca
portifolio1.dtypes[portifolio1.dtypes=='float64']

Series([], dtype: object)

**Estamos quase lá. Só falta retirarmos as colunas com um número muito elevado de valores nulos.**

## Retirando as variáveis com um número elevado de valores nulos

In [17]:
# Definindo a porcentagem de valores nulos de uma variável
porcent_nulos = 0.5

# Mostrando quantas colunas têm a porcentagem estipulada de valores nulos (ou mais) antes da retirada
len(portifolio1.isna().sum()[portifolio1.isna().sum() >= porcent_nulos * len(portifolio1)])

98

In [18]:
# Definindo uma função que elimina as colunas com um determinado 
#   percentual de valores nulos
def retira_colunas_com_porcent_nulos(df, porcent_nulos):
    colunas_com_muitos_nulos = [coluna for coluna in df.isna().sum()[df.isna().sum() >= porcent_nulos * len(df)].index]
    df.drop(colunas_com_muitos_nulos, axis=1, inplace=True)
    return df

In [19]:
# Aplicando a função recém criada no dataset portifolio1 e 
#   armazenando o seu retorno na própria variável
portifolio1 = retira_colunas_com_porcent_nulos(portifolio1, porcent_nulos)

In [20]:
# Mostrando quantas colunas têm a porcentagem estipulada de valores nulos (ou mais) depois da retirada
len(portifolio1.isna().sum()[portifolio1.isna().sum() >= porcent_nulos * len(portifolio1)])

0

**Com isso, o número de features foi reduzido.**

In [21]:
# Mostrando o tamanho do dataset depois do tratamento
portifolio1.shape

(555, 45)

**E as colunas restantes são as mostradas a seguir.**

In [22]:
# Mostrando os nomes das colunas restantes
[coluna for coluna in portifolio1.columns]

['id',
 'fl_matriz',
 'de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'setor',
 'idade_empresa_anos',
 'idade_emp_cat',
 'fl_me',
 'fl_sa',
 'fl_mei',
 'dt_situacao',
 'fl_email',
 'fl_telefone',
 'fl_rm',
 'nm_divisao',
 'nm_segmento',
 'fl_antt',
 'fl_veiculo',
 'fl_optante_simples',
 'vl_total_veiculos_pesados_grupo',
 'vl_total_veiculos_leves_grupo',
 'fl_optante_simei',
 'sg_uf_matriz',
 'de_saude_tributaria',
 'de_saude_rescencia',
 'nu_meses_rescencia',
 'de_nivel_atividade',
 'empsetorcensitariofaixarendapopulacao',
 'nm_meso_regiao',
 'nm_micro_regiao',
 'fl_passivel_iss',
 'qt_socios',
 'qt_socios_pf',
 'qt_socios_pj',
 'idade_media_socios',
 'idade_maxima_socios',
 'idade_minima_socios',
 'qt_socios_st_regular',
 'de_faixa_faturamento_estimado',
 'de_faixa_faturamento_estimado_grupo',
 'vl_faturamento_estimado_aux',
 'vl_faturamento_estimado_grupo_aux',
 'qt_filiais']

**Com esse tratamento, conseguimos eliminar boa parte das colunas que julgamos ser desnecessárias e podemos agora fazer o mesmo com o dataset <code>market</code> para tentar reduzí-lo.**

In [23]:
# Definido uma função apara fazer o tratamento no dataset de interesse
def trata_dados(df, porcent_nulos):

    # Chamando a função retira_colunas_de_valores_unicos()
    df = retira_colunas_de_valores_unicos(df)

    # Chamando a função troca_bool_para_numero()
    troca_bool_para_numero(df)

    # Chamando a função troca_64_para_32()
    df = reduz_memoria(df)

    # Chamando a função retira_colunas_com_muito_nulos()
    df = retira_colunas_com_porcent_nulos(df, porcent_nulos)

    # Retornando um DataFrame já tratado
    return df

In [24]:
# Mostrando as colunas com valores do tipo "boolean" antes do tratamento
market.dtypes[market.dtypes == 'bool']

fl_matriz         bool
fl_me             bool
fl_sa             bool
fl_epp            bool
fl_mei            bool
fl_ltda           bool
fl_st_especial    bool
fl_email          bool
fl_telefone       bool
dtype: object

In [25]:
# Mostrando as colunas com valores do tipo "int64" antes do tratamento
market.dtypes[market.dtypes=='int64']

qt_filiais    int64
dtype: object

In [26]:
# Mostrando as colunas com valores do tipo "float64" antes do tratamento
market.dtypes[market.dtypes=='float64']

idade_empresa_anos           float64
vl_total_tancagem            float64
vl_total_veiculos_antt       float64
vl_total_veiculos_leves      float64
vl_total_veiculos_pesados    float64
                              ...   
qt_funcionarios_12meses      float64
qt_funcionarios_24meses      float64
tx_crescimento_12meses       float64
tx_crescimento_24meses       float64
tx_rotatividade              float64
Length: 144, dtype: object

In [27]:
# Mostrando quantas colunas têm a porcentagem estipulada de valores nulos (ou mais) antes do tratamento
len(market.isna().sum()[market.isna().sum() >= porcent_nulos * len(market)])

131

In [28]:
# Mostrando novamente o tamanho do dataset "market" antes do tratamento
market.shape

(462298, 181)

In [29]:
%%time
# Chamando a função para fazer todo o tratamento do dataset de interesse
#   e armazenando o seu retorno na mesma variável
market = trata_dados(market, porcent_nulos)

Wall time: 2min 1s


In [30]:
# Mostrando as colunas com valores do tipo "float64" depois do tratamento
market.dtypes[market.dtypes=='float64']

Series([], dtype: object)

In [31]:
# Mostrando as colunas com valores do tipo "int64" depois do tratamento
market.dtypes[market.dtypes=='int64']

Series([], dtype: object)

In [32]:
# Mostrando quantas colunas têm a porcentagem estipulada de valores nulos (ou mais) depois do tratamento
len(market.isna().sum()[market.isna().sum() >= porcent_nulos * len(market)])

0

In [33]:
# Mostrando novamente o tamanho do dataset "market" depois do tratamento
market.shape

(462298, 49)

In [34]:
portifolio1.to_csv('portfolio1_parte_2.csv', index=False)
portifolio2.to_csv('portfolio2_parte_2.csv', index=False)
portifolio3.to_csv('portfolio3_parte_2.csv', index=False)
market.to_csv('market_parte_2.csv', index=False)